Our aim is to build Sunspot Prediction Model. We will be using Monthly Mean Total Sunspot Number, from 1749/01/01 to 2017/08/31 as our training data.

In [ ]:
import tensorflow as tf
print(tf.__version__)

# 1. Importing Libraries

In [ ]:
import os
import numpy as np 
import pandas as pd 
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

In [ ]:
tf.random.set_seed(9)
np.random.seed(9)

# 2. Loading Training Data

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
sunspots_df = pd.read_csv("/kaggle/input/sunspots/Sunspots.csv")
sunspots_df

# 3. Plotting Training Time Series

In [ ]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)

In [ ]:
series = np.array(sunspots_df[['Monthly Mean Total Sunspot Number']])
time = np.array(sunspots_df[['Unnamed: 0']])
plt.figure(figsize=(10, 6))
plot_series(time, series)

# 4. Splitting Training Data into Test and Validation Sets

In [ ]:
split_time = 2800
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

# 5. Windowed Dataset Function

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

# 6. Model Forecast Function

In [ ]:
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

# 7. Model
Let's use combination of CNN, LSTM and DNN in our model:

In [ ]:
tf.keras.backend.clear_session()

train_set = windowed_dataset(x_train, window_size=60, batch_size=180, shuffle_buffer=1000)

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=60, kernel_size=5,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=[None, 1]),
  tf.keras.layers.LSTM(60, return_sequences=True),
  tf.keras.layers.LSTM(60, return_sequences=True),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 400)
])


optimizer = tf.keras.optimizers.SGD(lr=2e-5, momentum=0.9)

model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])

# 8. Model Fitting

In [ ]:
history = model.fit(train_set,epochs=500)

# 9. Plotting Training Loss per epoch

In [ ]:
loss=history.history['loss']

epochs=range(len(loss))

plt.plot(epochs, loss, 'r')
plt.title('Training loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["Loss"])
plt.figure()


zoomed_loss = loss[200:]
zoomed_epochs = range(200,500)

plt.plot(zoomed_epochs, zoomed_loss, 'r')
plt.title('Training loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["Loss"])
plt.figure()

# 9. Model Forecast

In [ ]:
window_size = 60
rnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)
rnn_forecast = rnn_forecast[split_time - window_size:-1, -1, 0]

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(time_valid, x_valid)
plot_series(time_valid, rnn_forecast)
plt.legend(["Actual","Forecast"])

In [ ]:
#print(rnn_forecast)